# Problema di permutation flowshop

## Job scheduling con algoritmo genetico

Ci sono m macchine e n posti di lavoro. Ogni job contiene esattamente m operazioni. L'i-esima operazione del lavoro deve essere eseguita sulla i-esima macchina. Nessuna macchina può eseguire più di un'operazione contemporaneamente. Per ogni operazione di ogni lavoro, viene specificato il tempo di esecuzione.
Le operazioni all'interno di un lavoro devono essere eseguite nell'ordine specificato. La prima operazione viene eseguita sulla prima macchina, poi (quando la prima operazione è terminata) la seconda operazione sulla seconda macchina, e così via fino all'ennesima operazione. Tuttavia, i lavori possono essere eseguiti in qualsiasi ordine.
L'obiettivo è quello di completare tutti i job nel minore tempo possibile, ovvero minimizzare il makespan.

### Ipotesi iniziali
Si considera un problema dove ci sono m macchine indipendenti, scheduling non preemptive e tempo di rilascio nullo per tutti i job, quindi i job posso essere eseguiti in qualsiasi istante e senza interruzioni.
Il modello di scheduling può essere modellato nel modo seguente:
F_m || C_max

In [1]:
from functions import *
import numpy as np
import re

population_number: int = 60
iterations_number: int = 35 # numero di generazioni
p_mutation: float = 0.05    # probabilità di mutazione

bench_data = []             # matrice tempi di esecuzione job-macchina

In [2]:
dictionary = {}
file = open("taillard_50x20.txt", "rt")
with file as f:
    for line in f:
       (key, val) = line.split(':')
       dictionary[key] = val    
print(dictionary)
file.close()

{'number of jobs': '50\n', 'number of machines': '20\n', 'processing times': ' [52 95 42 75 44 57 89 53 84 62 91 14 95 89 4 95 2 97 68 20 33 51 98 8 85 86 73 4 40 98 12 59 44 46 2 41 28 83 28 21 80 71 4 60 34 55 53 96 37 37][63 99 69 70 53 21 10 31 80 18 5 18 17 71 90 93 14 49 52 7 78 57 41 75 98 93 33 75 68 33 60 82 24 99 4 97 24 50 55 91 46 58 17 47 82 6 15 91 74 42][82 21 79 95 46 23 40 95 87 37 24 24 65 62 19 67 66 6 65 59 2 67 82 90 30 63 5 93 53 85 81 73 34 74 13 78 35 20 16 48 12 11 80 9 24 76 32 35 66 48][16 26 46 66 76 31 36 8 37 21 3 76 67 5 47 72 66 56 95 49 47 26 81 56 76 66 36 53 26 52 29 36 68 21 71 61 71 69 28 86 27 41 86 55 17 62 96 59 53 93][63 55 59 35 21 59 78 25 30 38 78 79 58 44 38 76 70 72 85 8 10 84 42 67 20 24 75 23 33 60 20 75 83 26 92 29 39 14 74 66 86 10 27 8 7 97 84 56 61 9][94 34 89 62 47 66 76 15 18 54 24 55 96 10 12 96 53 92 77 6 91 14 41 30 85 17 23 60 76 39 85 10 65 15 55 41 28 93 88 27 77 81 19 76 55 67 65 8 18 56][79 21 93 32 8 45 37 78 26 98 17 25 21

In [3]:
n_jobs = int(dictionary['number of jobs'])
n_machines = int(dictionary['number of machines'])
print(n_jobs)
print(n_machines)

50
20


In [4]:

processing_times = re.findall('\[(.*?)\]', dictionary['processing times'])
print(processing_times)

['52 95 42 75 44 57 89 53 84 62 91 14 95 89 4 95 2 97 68 20 33 51 98 8 85 86 73 4 40 98 12 59 44 46 2 41 28 83 28 21 80 71 4 60 34 55 53 96 37 37', '63 99 69 70 53 21 10 31 80 18 5 18 17 71 90 93 14 49 52 7 78 57 41 75 98 93 33 75 68 33 60 82 24 99 4 97 24 50 55 91 46 58 17 47 82 6 15 91 74 42', '82 21 79 95 46 23 40 95 87 37 24 24 65 62 19 67 66 6 65 59 2 67 82 90 30 63 5 93 53 85 81 73 34 74 13 78 35 20 16 48 12 11 80 9 24 76 32 35 66 48', '16 26 46 66 76 31 36 8 37 21 3 76 67 5 47 72 66 56 95 49 47 26 81 56 76 66 36 53 26 52 29 36 68 21 71 61 71 69 28 86 27 41 86 55 17 62 96 59 53 93', '63 55 59 35 21 59 78 25 30 38 78 79 58 44 38 76 70 72 85 8 10 84 42 67 20 24 75 23 33 60 20 75 83 26 92 29 39 14 74 66 86 10 27 8 7 97 84 56 61 9', '94 34 89 62 47 66 76 15 18 54 24 55 96 10 12 96 53 92 77 6 91 14 41 30 85 17 23 60 76 39 85 10 65 15 55 41 28 93 88 27 77 81 19 76 55 67 65 8 18 56', '79 21 93 32 8 45 37 78 26 98 17 25 21 28 68 24 62 89 60 64 38 90 87 1 99 34 9 22 74 14 14 84 75 37 32 2

In [5]:
i = 1
for machine_times in processing_times:
    list_times = machine_times.split(' ')
    list_times = [int(x) for x in list_times]
    bench_data.append(list_times)
    print('machine' + str(i) + ': ')
    print(list_times)
    i = i + 1
print('\n')
print('bench_data: ')
print(bench_data)

machine1: 
[52, 95, 42, 75, 44, 57, 89, 53, 84, 62, 91, 14, 95, 89, 4, 95, 2, 97, 68, 20, 33, 51, 98, 8, 85, 86, 73, 4, 40, 98, 12, 59, 44, 46, 2, 41, 28, 83, 28, 21, 80, 71, 4, 60, 34, 55, 53, 96, 37, 37]
machine2: 
[63, 99, 69, 70, 53, 21, 10, 31, 80, 18, 5, 18, 17, 71, 90, 93, 14, 49, 52, 7, 78, 57, 41, 75, 98, 93, 33, 75, 68, 33, 60, 82, 24, 99, 4, 97, 24, 50, 55, 91, 46, 58, 17, 47, 82, 6, 15, 91, 74, 42]
machine3: 
[82, 21, 79, 95, 46, 23, 40, 95, 87, 37, 24, 24, 65, 62, 19, 67, 66, 6, 65, 59, 2, 67, 82, 90, 30, 63, 5, 93, 53, 85, 81, 73, 34, 74, 13, 78, 35, 20, 16, 48, 12, 11, 80, 9, 24, 76, 32, 35, 66, 48]
machine4: 
[16, 26, 46, 66, 76, 31, 36, 8, 37, 21, 3, 76, 67, 5, 47, 72, 66, 56, 95, 49, 47, 26, 81, 56, 76, 66, 36, 53, 26, 52, 29, 36, 68, 21, 71, 61, 71, 69, 28, 86, 27, 41, 86, 55, 17, 62, 96, 59, 53, 93]
machine5: 
[63, 55, 59, 35, 21, 59, 78, 25, 30, 38, 78, 79, 58, 44, 38, 76, 70, 72, 85, 8, 10, 84, 42, 67, 20, 24, 75, 23, 33, 60, 20, 75, 83, 26, 92, 29, 39, 14, 74, 66

In [6]:
processing_time = [] # tempi di esecuzione del job j sulla macchina i
for i in range(n_jobs):
    temp = []
    for j in range(n_machines):
        temp.append(bench_data[j][i])
    processing_time.append(temp)


print(processing_time)

[[52, 63, 82, 16, 63, 94, 79, 22, 80, 96, 53, 54, 71, 27, 95, 3, 92, 80, 61, 74], [95, 99, 21, 26, 55, 34, 21, 6, 13, 3, 19, 67, 90, 93, 53, 80, 62, 38, 86, 38], [42, 69, 79, 46, 59, 89, 93, 24, 64, 50, 99, 25, 59, 49, 54, 78, 11, 55, 16, 4], [75, 70, 95, 66, 35, 62, 32, 55, 77, 57, 62, 77, 82, 63, 22, 32, 83, 34, 42, 31], [44, 53, 46, 76, 21, 47, 8, 48, 17, 66, 88, 38, 22, 65, 84, 53, 87, 85, 14, 62], [57, 21, 23, 31, 59, 66, 45, 57, 78, 84, 93, 98, 88, 34, 54, 43, 66, 44, 92, 39], [89, 10, 40, 36, 78, 76, 37, 78, 82, 98, 34, 96, 35, 10, 2, 85, 98, 47, 67, 97], [53, 31, 95, 8, 25, 15, 78, 5, 4, 55, 72, 20, 49, 56, 80, 19, 42, 66, 77, 57], [84, 80, 87, 37, 30, 18, 26, 50, 72, 70, 42, 15, 78, 51, 84, 48, 23, 19, 46, 9], [62, 18, 37, 21, 38, 54, 98, 83, 93, 32, 65, 36, 69, 97, 66, 49, 45, 66, 41, 54], [91, 5, 24, 3, 78, 24, 17, 70, 68, 31, 39, 65, 76, 52, 25, 66, 52, 61, 78, 13], [14, 18, 24, 76, 79, 55, 25, 21, 25, 64, 79, 97, 2, 46, 16, 22, 6, 60, 3, 47], [95, 17, 65, 67, 58, 96, 21, 7

In [7]:
# creazione popolazione
population = initialize_population(population_number, n_jobs)
#print(population)

for evaluation in range(iterations_number):
    # Select parents
    parents = get_parents(population, processing_time, n_jobs, n_machines)
    childs = []
    # operatore di crossover per generare nuove sequenze di scheduling
    childs.append(crossover(parents))
   # print('\n')
   # print(childs)

    # operatore di mutazione
    mutated_childs = []
    for child in childs:
        r = np.random.rand()
        if r < p_mutation:
            mutated_child = mutation(child)
            mutated_childs.append(mutated_child)

    childs.extend(mutated_childs)
    if len(childs) > 0:
        update_population(population, childs, processing_time, n_jobs, n_machines)


fitness totale: 
15565
cumulated_fitnesses:
[(0.023128814648249278, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.038869257950530034, [43, 29, 36, 32, 17, 21, 4, 48, 7, 2, 39, 11, 9, 28, 27, 18, 1, 37, 38, 45, 16, 3, 44, 30, 46, 24, 42, 47, 41, 26, 19, 22, 12, 8, 20, 34, 0, 14, 23, 5, 40, 33, 13, 31, 35, 49, 25, 6, 10, 15]), (0.05248955991005461, [15, 43, 30, 31, 19, 35, 6, 41, 28, 26, 32, 21, 40, 39, 25, 36, 12, 8, 2, 20, 4, 27, 48, 46, 9, 18, 11, 33, 22, 44, 23, 3, 45, 29, 47, 24, 49, 5, 16, 14, 1, 13, 42, 37, 7, 34, 10, 38, 17, 0]), (0.05987793125602313, [43, 20, 26, 11, 24, 8, 13, 12, 42, 32, 9, 17, 21, 35, 6, 19, 44, 18, 41, 30, 2, 14, 38, 37, 5, 40, 34, 7, 16, 46, 49, 39, 36, 33, 25, 45, 27, 22, 3, 10, 31, 48, 29, 4, 47, 15, 1, 28, 0, 23]), (0.07073562479922904, [6, 22, 16, 29, 38, 17, 31, 19, 10, 15, 28, 12, 35, 39, 18, 4, 34, 40, 

fitness totale: 
10476
cumulated_fitnesses:
[(0.02453226422298587, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.038087056128293245, [43, 29, 36, 32, 17, 21, 4, 48, 7, 2, 39, 11, 9, 28, 27, 18, 1, 37, 38, 45, 16, 3, 44, 30, 46, 24, 42, 47, 41, 26, 19, 22, 12, 8, 20, 34, 0, 14, 23, 5, 40, 33, 13, 31, 35, 49, 25, 6, 10, 15]), (0.048491790759831996, [15, 43, 30, 31, 19, 35, 6, 41, 28, 26, 32, 21, 40, 39, 25, 36, 12, 8, 2, 20, 4, 27, 48, 46, 9, 18, 11, 33, 22, 44, 23, 3, 45, 29, 47, 24, 49, 5, 16, 14, 1, 13, 42, 37, 7, 34, 10, 38, 17, 0]), (0.049637266132111495, [43, 20, 26, 11, 24, 8, 13, 12, 42, 32, 9, 17, 21, 35, 6, 19, 44, 18, 41, 30, 2, 14, 38, 37, 5, 40, 34, 7, 16, 46, 49, 39, 36, 33, 25, 45, 27, 22, 3, 10, 31, 48, 29, 4, 47, 15, 1, 28, 0, 23]), (0.05593738067964872, [6, 22, 16, 29, 38, 17, 31, 19, 10, 15, 28, 12, 35, 39, 18, 4, 34, 40,

fitness totale: 
9881
cumulated_fitnesses:
[(0.024795061228620584, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.0379516243295213, [43, 29, 36, 32, 17, 21, 4, 48, 7, 2, 39, 11, 9, 28, 27, 18, 1, 37, 38, 45, 16, 3, 44, 30, 46, 24, 42, 47, 41, 26, 19, 22, 12, 8, 20, 34, 0, 14, 23, 5, 40, 33, 13, 31, 35, 49, 25, 6, 10, 15]), (0.04776844448942415, [15, 43, 30, 31, 19, 35, 6, 41, 28, 26, 32, 21, 40, 39, 25, 36, 12, 8, 2, 20, 4, 27, 48, 46, 9, 18, 11, 33, 22, 44, 23, 3, 45, 29, 47, 24, 49, 5, 16, 14, 1, 13, 42, 37, 7, 34, 10, 38, 17, 0]), (0.04776844448942415, [43, 20, 26, 11, 24, 8, 13, 12, 42, 32, 9, 17, 21, 35, 6, 19, 44, 18, 41, 30, 2, 14, 38, 37, 5, 40, 34, 7, 16, 46, 49, 39, 36, 33, 25, 45, 27, 22, 3, 10, 31, 48, 29, 4, 47, 15, 1, 28, 0, 23]), (0.05323347839287521, [6, 22, 16, 29, 38, 17, 31, 19, 10, 15, 28, 12, 35, 39, 18, 4, 34, 40, 9, 

fitness totale: 
9367
cumulated_fitnesses:
[(0.024981317390840183, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.03768549161951532, [43, 29, 36, 32, 17, 21, 4, 48, 7, 2, 39, 11, 9, 28, 27, 18, 1, 37, 38, 45, 16, 3, 44, 30, 46, 24, 42, 47, 41, 26, 19, 22, 12, 8, 20, 34, 0, 14, 23, 5, 40, 33, 13, 31, 35, 49, 25, 6, 10, 15]), (0.04686665954948222, [15, 43, 30, 31, 19, 35, 6, 41, 28, 26, 32, 21, 40, 39, 25, 36, 12, 8, 2, 20, 4, 27, 48, 46, 9, 18, 11, 33, 22, 44, 23, 3, 45, 29, 47, 24, 49, 5, 16, 14, 1, 13, 42, 37, 7, 34, 10, 38, 17, 0]), (0.05145724351446568, [6, 22, 16, 29, 38, 17, 31, 19, 10, 15, 28, 12, 35, 39, 18, 4, 34, 40, 9, 44, 8, 7, 11, 14, 25, 24, 45, 46, 43, 30, 36, 42, 49, 23, 32, 47, 48, 37, 5, 41, 21, 3, 27, 33, 2, 20, 0, 13, 1, 26]), (0.08989003950037365, [23, 32, 8, 47, 46, 28, 19, 29, 48, 24, 15, 9, 39, 12, 16, 37, 20, 22, 0,

fitness totale: 
6059
cumulated_fitnesses:
[(0.026241954117841228, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.03350387852780987, [43, 29, 36, 32, 17, 21, 4, 48, 7, 2, 39, 11, 9, 28, 27, 18, 1, 37, 38, 45, 16, 3, 44, 30, 46, 24, 42, 47, 41, 26, 19, 22, 12, 8, 20, 34, 0, 14, 23, 5, 40, 33, 13, 31, 35, 49, 25, 6, 10, 15]), (0.03531935963030203, [15, 43, 30, 31, 19, 35, 6, 41, 28, 26, 32, 21, 40, 39, 25, 36, 12, 8, 2, 20, 4, 27, 48, 46, 9, 18, 11, 33, 22, 44, 23, 3, 45, 29, 47, 24, 49, 5, 16, 14, 1, 13, 42, 37, 7, 34, 10, 38, 17, 0]), (0.082356824558508, [23, 32, 8, 47, 46, 28, 19, 29, 48, 24, 15, 9, 39, 12, 16, 37, 20, 22, 0, 33, 13, 14, 43, 45, 34, 3, 10, 30, 18, 11, 38, 25, 36, 6, 31, 35, 4, 40, 41, 27, 44, 21, 17, 26, 2, 1, 49, 42, 7, 5]), (0.10678329757385707, [19, 34, 24, 44, 47, 27, 0, 29, 48, 45, 32, 39, 30, 35, 21, 36, 10, 41, 18,

...

fitness totale: 
6143
cumulated_fitnesses:
[(0.024092462965977536, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.029464431059742797, [43, 29, 36, 32, 17, 21, 4, 48, 7, 2, 39, 11, 9, 28, 27, 18, 1, 37, 38, 45, 16, 3, 44, 30, 46, 24, 42, 47, 41, 26, 19, 22, 12, 8, 20, 34, 0, 14, 23, 5, 40, 33, 13, 31, 35, 49, 25, 6, 10, 15]), (0.0740680449291877, [23, 32, 8, 47, 46, 28, 19, 29, 48, 24, 15, 9, 39, 12, 16, 37, 20, 22, 0, 33, 13, 14, 43, 45, 34, 3, 10, 30, 18, 11, 38, 25, 36, 6, 31, 35, 4, 40, 41, 27, 44, 21, 17, 26, 2, 1, 49, 42, 7, 5]), (0.09636985186391014, [19, 34, 24, 44, 47, 27, 0, 29, 48, 45, 32, 39, 30, 35, 21, 36, 10, 41, 18, 9, 31, 46, 5, 23, 28, 40, 25, 37, 11, 49, 17, 22, 13, 26, 38, 6, 2, 20, 12, 7, 1, 16, 33, 8, 42, 3, 4, 15, 14, 43]), (0.10971837864235716, [5, 23, 14, 13, 19, 6, 35, 8, 36, 38, 34, 9, 3, 26, 49, 37, 28, 30, 2, 12

fitness totale: 
5654
cumulated_fitnesses:
[(0.023876901308807923, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.02741422002122391, [43, 29, 36, 32, 17, 21, 4, 48, 7, 2, 39, 11, 9, 28, 27, 18, 1, 37, 38, 45, 16, 3, 44, 30, 46, 24, 42, 47, 41, 26, 19, 22, 12, 8, 20, 34, 0, 14, 23, 5, 40, 33, 13, 31, 35, 49, 25, 6, 10, 15]), (0.07357622921825256, [23, 32, 8, 47, 46, 28, 19, 29, 48, 24, 15, 9, 39, 12, 16, 37, 20, 22, 0, 33, 13, 14, 43, 45, 34, 3, 10, 30, 18, 11, 38, 25, 36, 6, 31, 35, 4, 40, 41, 27, 44, 21, 17, 26, 2, 1, 49, 42, 7, 5]), (0.09550760523523169, [19, 34, 24, 44, 47, 27, 0, 29, 48, 45, 32, 39, 30, 35, 21, 36, 10, 41, 18, 9, 31, 46, 5, 23, 28, 40, 25, 37, 11, 49, 17, 22, 13, 26, 38, 6, 2, 20, 12, 7, 1, 16, 33, 8, 42, 3, 4, 15, 14, 43]), (0.10771135479306686, [5, 23, 14, 13, 19, 6, 35, 8, 36, 38, 34, 9, 3, 26, 49, 37, 28, 30, 2, 12

fitness totale: 
5713
cumulated_fitnesses:
[(0.0218799229826711, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.02363031682128479, [43, 29, 36, 32, 17, 21, 4, 48, 7, 2, 39, 11, 9, 28, 27, 18, 1, 37, 38, 45, 16, 3, 44, 30, 46, 24, 42, 47, 41, 26, 19, 22, 12, 8, 20, 34, 0, 14, 23, 5, 40, 33, 13, 31, 35, 49, 25, 6, 10, 15]), (0.06756520217048836, [23, 32, 8, 47, 46, 28, 19, 29, 48, 24, 15, 9, 39, 12, 16, 37, 20, 22, 0, 33, 13, 14, 43, 45, 34, 3, 10, 30, 18, 11, 38, 25, 36, 6, 31, 35, 4, 40, 41, 27, 44, 21, 17, 26, 2, 1, 49, 42, 7, 5]), (0.0875196919306844, [19, 34, 24, 44, 47, 27, 0, 29, 48, 45, 32, 39, 30, 35, 21, 36, 10, 41, 18, 9, 31, 46, 5, 23, 28, 40, 25, 37, 11, 49, 17, 22, 13, 26, 38, 6, 2, 20, 12, 7, 1, 16, 33, 8, 42, 3, 4, 15, 14, 43]), (0.09784701557850517, [5, 23, 14, 13, 19, 6, 35, 8, 36, 38, 34, 9, 3, 26, 49, 37, 28, 30, 2, 12, 4

fitness totale: 
6141
cumulated_fitnesses:
[(0.01840091190359876, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.0573196547793519, [23, 32, 8, 47, 46, 28, 19, 29, 48, 24, 15, 9, 39, 12, 16, 37, 20, 22, 0, 33, 13, 14, 43, 45, 34, 3, 10, 30, 18, 11, 38, 25, 36, 6, 31, 35, 4, 40, 41, 27, 44, 21, 17, 26, 2, 1, 49, 42, 7, 5]), (0.07392932747109592, [19, 34, 24, 44, 47, 27, 0, 29, 48, 45, 32, 39, 30, 35, 21, 36, 10, 41, 18, 9, 31, 46, 5, 23, 28, 40, 25, 37, 11, 49, 17, 22, 13, 26, 38, 6, 2, 20, 12, 7, 1, 16, 33, 8, 42, 3, 4, 15, 14, 43]), (0.08158280410356619, [5, 23, 14, 13, 19, 6, 35, 8, 36, 38, 34, 9, 3, 26, 49, 37, 28, 30, 2, 12, 45, 41, 43, 39, 15, 40, 25, 44, 10, 29, 46, 20, 4, 16, 1, 27, 48, 47, 18, 32, 33, 31, 7, 24, 17, 0, 22, 21, 11, 42]), (0.08174564403191663, [29, 5, 18, 23, 2, 43, 13, 42, 35, 0, 41, 34, 24, 10, 33, 49, 38, 6, 40, 26

fitness totale: 
6251
cumulated_fitnesses:
[(0.01727723564229723, [9, 36, 33, 26, 6, 42, 37, 27, 1, 39, 38, 22, 14, 13, 25, 12, 11, 19, 8, 20, 24, 23, 2, 44, 16, 28, 43, 5, 29, 30, 45, 31, 10, 0, 4, 34, 49, 17, 41, 15, 35, 40, 3, 7, 47, 32, 21, 18, 48, 46]), (0.0547112462006079, [23, 32, 8, 47, 46, 28, 19, 29, 48, 24, 15, 9, 39, 12, 16, 37, 20, 22, 0, 33, 13, 14, 43, 45, 34, 3, 10, 30, 18, 11, 38, 25, 36, 6, 31, 35, 4, 40, 41, 27, 44, 21, 17, 26, 2, 1, 49, 42, 7, 5]), (0.07022876339785634, [19, 34, 24, 44, 47, 27, 0, 29, 48, 45, 32, 39, 30, 35, 21, 36, 10, 41, 18, 9, 31, 46, 5, 23, 28, 40, 25, 37, 11, 49, 17, 22, 13, 26, 38, 6, 2, 20, 12, 7, 1, 16, 33, 8, 42, 3, 4, 15, 14, 43]), (0.07694768836986082, [5, 23, 14, 13, 19, 6, 35, 8, 36, 38, 34, 9, 3, 26, 49, 37, 28, 30, 2, 12, 45, 41, 43, 39, 15, 40, 25, 44, 10, 29, 46, 20, 4, 16, 1, 27, 48, 47, 18, 32, 33, 31, 7, 24, 17, 0, 22, 21, 11, 42]), (0.09166533354663253, [0, 2, 45, 21, 14, 43, 4, 39, 27, 42, 44, 49, 23, 3, 26, 13, 37, 25, 11, 9,

In [8]:
makespans = []
for individual in population:
    ind_makespan = (calc_makespan(individual, processing_time, n_jobs, n_machines), individual)
    makespans.append(ind_makespan)
makespans.sort(key=lambda x: x[0])

best_seq = list(map(int, makespans[0][1]))
best_makespan = get_makespan(n_machines, best_seq, bench_data)

In [9]:
print(makespans)

[(4610, [16, 35, 9, 49, 31, 5, 48, 29, 44, 8, 38, 20, 24, 7, 3, 1, 12, 0, 41, 21, 34, 10, 46, 25, 23, 36, 45, 15, 40, 42, 26, 43, 32, 4, 47, 6, 2, 39, 19, 22, 37, 13, 33, 30, 28, 11, 27, 18, 17, 14]), (4631, [5, 14, 13, 19, 35, 8, 38, 9, 49, 37, 28, 30, 24, 7, 3, 1, 12, 0, 41, 21, 34, 10, 46, 25, 23, 36, 45, 15, 40, 42, 26, 43, 32, 4, 47, 6, 2, 39, 44, 29, 20, 16, 27, 48, 18, 33, 31, 17, 22, 11]), (4633, [23, 14, 19, 8, 36, 38, 3, 37, 41, 43, 39, 15, 21, 6, 13, 2, 1, 22, 34, 31, 45, 40, 5, 42, 0, 33, 35, 27, 20, 28, 30, 9, 12, 46, 29, 49, 18, 26, 25, 44, 10, 4, 16, 48, 47, 32, 7, 24, 17, 11]), (4638, [23, 32, 8, 47, 46, 28, 19, 29, 48, 24, 15, 9, 39, 12, 16, 37, 20, 22, 0, 33, 13, 14, 43, 45, 34, 3, 10, 30, 18, 11, 38, 25, 36, 6, 31, 35, 4, 40, 41, 27, 44, 21, 17, 26, 2, 1, 49, 42, 7, 5]), (4648, [30, 39, 46, 25, 22, 42, 4, 5, 7, 35, 6, 48, 45, 36, 23, 13, 14, 49, 16, 17, 1, 15, 32, 11, 34, 44, 27, 40, 33, 8, 0, 3, 41, 28, 20, 26, 43, 10, 47, 9, 19, 37, 12, 18, 21, 38, 2, 24, 29, 31]),

In [10]:
print(best_makespan)

4625


In [11]:
print(best_seq)

[16, 35, 9, 49, 31, 5, 48, 29, 44, 8, 38, 20, 24, 7, 3, 1, 12, 0, 41, 21, 34, 10, 46, 25, 23, 36, 45, 15, 40, 42, 26, 43, 32, 4, 47, 6, 2, 39, 19, 22, 37, 13, 33, 30, 28, 11, 27, 18, 17, 14]
